In [45]:
%matplotlib inline
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
torch.set_printoptions(edgeitems=2)

In [46]:
# CSVファイルの読み込み
df = pd.read_csv('xy(1).csv')

# データの抽出
x = df['x'].values
y = df['y'].values

x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [47]:
def cross_product(v1, v2):
    return torch.cross(v1, v2, dim=0)

def magnetic(axis, vec, theta):
    cos_theta = torch.cos(theta)
    sin_theta = torch.sin(theta)
    
    axis = axis.unsqueeze(0)  # (1, 3)
    vec = vec.unsqueeze(0)    # (1, 3)
    
    rotated_vecs = []
    for i in range(theta.size(0)):
        theta_i = theta[i]
        cos_theta_i = cos_theta[i]
        sin_theta_i = sin_theta[i]
        
        axis_i = axis[0]
        vec_i = vec[0]
        
        rotated_vec = vec_i * cos_theta_i + cross_product(axis_i, vec_i) * sin_theta_i + axis_i * (torch.matmul(axis_i.unsqueeze(1).T, vec_i.unsqueeze(1)).squeeze() * (1 - cos_theta_i))
        rotated_vecs.append(rotated_vec)
    
    return torch.stack(rotated_vecs)

def model(x, params):
    rad = torch.pi / 180.0  # Conversion factor from degrees to radians
    nx = torch.cos(params[7] * rad) * torch.sin(params[6] * rad)
    ny = torch.sin(params[7] * rad) * torch.sin(params[6] * rad)
    nz = torch.cos(params[6] * rad)
    sx = torch.cos(params[9] * rad) * torch.sin(params[8] * rad)
    sy = torch.sin(params[9] * rad) * torch.sin(params[8] * rad)
    sz = torch.cos(params[8] * rad)
    theta = x * rad
    
    h = magnetic(torch.tensor([nx, ny, nz], requires_grad=True), torch.tensor([sx, sy, sz], requires_grad=True), theta)
    K = torch.tensor([[params[0], params[5], params[4]], 
                      [params[5], params[1], params[3]], 
                      [params[4], params[3], params[2]]], requires_grad=True)
    
    # 63個のスカラー結果を得るための計算
    K1 = torch.einsum('ij,jk,ik->i', h, K, h)
    return K1

In [48]:
def loss_fn(y_p, y):
    squared_diffs = (y_p - y)**2
    return squared_diffs.mean()

In [49]:
params = [torch.tensor(1.0, requires_grad=True) for _ in range(10)]

In [50]:
# 各パラメータの勾配を確認する
for i, param in enumerate(params):
    if param.grad is not None:
        print(f'param_{i} gradient: {param.grad}')
    else:
        print(f'param_{i} has no gradient')

param_0 has no gradient
param_1 has no gradient
param_2 has no gradient
param_3 has no gradient
param_4 has no gradient
param_5 has no gradient
param_6 has no gradient
param_7 has no gradient
param_8 has no gradient
param_9 has no gradient


In [51]:
params.grad is None

AttributeError: 'list' object has no attribute 'grad'

In [ ]:
loss = loss_fn(model(x, *params), y)
loss.backward()

params.grad

TypeError: model() takes 2 positional arguments but 11 were given

In [ ]:
if params.grad is not None:
    params.grad.zero_()

In [ ]:
def training_loop(n_epochs, learning_rate, params, x, y):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # <1>
            params.grad.zero_()
        
        y_p = model(x, *params) 
        loss = loss_fn(x, y)
        loss.backward()
        
        with torch.no_grad():  # <2>
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [ ]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = [torch.tensor(1.0, requires_grad=True) for _ in range(10)], # <1> 
    x = x, # <2> 
    y = y)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)